In [180]:
from IPython import get_ipython
get_ipython().magic('reset -sf')

import numpy as np
import pandas as pd

pd.set_option('display.max_columns', 30) 
pd.set_option('display.max_colwidth', -1) 

from time import time
from datetime import datetime

import os
cwd = os.path.abspath(os.path.curdir) 

In [181]:
# load the data from csv
#----------------------------------------
def load_data(filename):
#----------------------------------------

    t1 = time()
    print ('Starting Data Load...')
    
    df=pd.read_csv(filename, sep=',', parse_dates=True) 
    df.Date = pd.to_datetime(df.Date, yearfirst=True)
       
    print ('Data Loaded:', df.shape, 'in', round(time()-t1,4), 'seconds')
        
    return df.iloc[:,:]                # return PD dataframe     
   
#----------------------------------------

In [182]:
def make_corrs(df,dataset):
    start_dt=df.iloc[0,0]
    end_dt=df.iloc[-1,0]+pd.Timedelta(days=15)
    months=pd.date_range(start=start_dt, end=end_dt, freq='M')

    x=len(months)        # number of corr matrices
    y=df.shape[1]-1      # dimension of corr matrix (don't include date column)

    corr_mats_np = np.zeros([x,y,y])
    dates=[]
    nodes=df.columns.values[1:]

    frm = start_dt
    for i,d in enumerate(months):
        corr_mats_np[i,:,:]=df[(df.Date>frm) & (df.Date<=d)].corr().values
        dates.append(d)
        frm=d

    if dataset == 'sectors':
        nodes_descr=[('TRANS','Transport'),('REAL','Real Estate'),('MTRL','Material'),('ENER','Energy'),
                     ('FIN','Finance'),('INDU','Industrial'),('TECH','Technology'),('CONSS','Consumer Staple'),
                     ('UTIL','Utilities'),('HLTH','Health Care'),('CONSD','Consumer Discret.')]
    
    np.save(dataset+'_corr.npy', corr_mats_np)
    np.save(dataset+'_dates.npy', np.array(dates))
    np.save(dataset+'_nodes.npy', np.array(nodes))
    np.save(dataset+'_nodes_descr.npy', np.array(nodes_descr))
    

In [183]:
dataset='sectors'
loadcsv='sectors.csv'
df = load_data(loadcsv)
make_corrs(df,dataset)

Starting Data Load...
Data Loaded: (2518, 12) in 0.2187 seconds


In [186]:
dataset='sectors'
corr_mats = np.load(dataset+'_corr.npy')
dates = np.load(dataset+'_dates.npy')
nodes = np.load(dataset+'_nodes.npy')
nodes_descr = np.load(dataset+'_nodes_descr.npy')

In [197]:
#np.sum((corr_mats< 0))
#np.sum((corr_mats> 0))
np.std(corr_mats)

0.27354821724128114